# 1. Read the data

In [46]:
import pandas as pd
import numpy as np

In [47]:
data = pd.read_csv('data/beautyProducts.csv')
data.head()

,update_time,id,title,price,sale_count,comment_count,店名
0,2016/11/14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂
1,2016/11/14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂
2,2016/11/14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂
3,2016/11/14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂
4,2016/11/14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂


In [48]:
data.shape

(27598, 7)

In [49]:
# Check the information of each field
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27598 entries, 0 to 27597
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   update_time    27598 non-null  object 
 1   id             27598 non-null  object 
 2   title          27598 non-null  object 
 3   price          27598 non-null  float64
 4   sale_count     25244 non-null  float64
 5   comment_count  25244 non-null  float64
 6   店名             27598 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1.5+ MB


In [50]:
# Check the specific values of the store name field
data['店名'].value_counts()

店名
悦诗风吟    3021
佰草集     2265
欧莱雅     1974
雅诗兰黛    1810
倩碧      1704
美加净     1678
欧珀莱     1359
妮维雅     1329
相宜本草    1313
兰蔻      1285
娇兰      1193
自然堂     1190
玉兰油     1135
兰芝      1091
美宝莲      825
资生堂      821
植村秀      750
薇姿       746
雅漾       663
雪花秀      543
SKII     469
蜜丝佛陀     434
Name: count, dtype: int64

# 2. Data cleaning

## 2.1 Handling duplicate data

In [51]:
# Remove duplicate data
data = data.drop_duplicates(inplace=False) # hash

In [52]:
# Check the dimensional changes of the dataset
data.shape # 27598-27512 = 86

(27512, 7)

In [53]:
# Reset the DataFrame index
data.reset_index(inplace=True,drop=True)

## 2.2 Handling missing values

In [54]:
# Check for missing values
data.isnull().any()

update_time      False
id               False
title            False
price            False
sale_count        True
comment_count     True
店名               False
dtype: bool

In [55]:
# View the data structure
data.describe()

,price,sale_count,comment_count
count,27512.000000,2.516200e+04,25162.000000
mean,363.423512,1.231605e+04,1121.741197
std,614.876153,5.241236e+04,5277.781581
min,1.000000,0.000000e+00,0.000000
25%,99.000000,2.780000e+02,21.000000
50%,205.000000,1.443000e+03,153.000000
75%,390.000000,6.353000e+03,669.000000
max,11100.000000,1.923160e+06,202930.000000


In [56]:
# Check the mode of the sale_count column
mode_01 = data.sale_count.mode()
mode_01

0    0.0
Name: sale_count, dtype: float64

In [57]:
# the comment_count column
mode_02 = data.comment_count.mode()
mode_02

0    0.0
Name: comment_count, dtype: float64

In [58]:
# The mode of both columns is 0, and according to the meaning of the labeled data, it is possible for sales volume and the number of reviews to be 0. 
# Therefore, 0 is used to fill in the missing values.
data = data.fillna(0)

In [59]:
data.isnull().sum()

update_time      0
id               0
title            0
price            0
sale_count       0
comment_count    0
店名               0
dtype: int64

## 2.3 Extract useful information from the table and add it as new columns

### (1) Perform word segmentation on product titles

In [60]:
# Jieba Tokenizer: It has a huge built-in lexicon and calculates the segmentation method with the highest probability through algorithms.
import jieba
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [61]:
# Load the data from the title column in the above dataset and perform Chinese word segmentation on it
title_cut = []
for i in data.title:
    j = jieba.lcut(i) # Word segmentation processing
    title_cut.append(j) # The result of word segmentation is put into the title_cut set.

In [62]:
data['item_name_cut'] = title_cut

In [63]:
data[['title','item_name_cut']].head()

,title,item_name_cut
0,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,"[CHANDO, /, 自然, 堂, , 雪域, 精粹, 纯粹, 滋润霜, 50g, ,..."
1,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,"[CHANDO, /, 自然, 堂, 凝, 时鲜, 颜肌活, 乳液, 120ML, , 淡..."
2,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,"[CHANDO, /, 自然, 堂, 活泉, 保湿, 修护, 精华, 水, （, 滋润, 型..."
3,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,"[CHANDO, /, 自然, 堂, , 男士, 劲爽, 控油, 洁面膏, , 100g..."
4,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,"[CHANDO, /, 自然, 堂, 雪域, 精粹, 纯粹, 滋润霜, （, 清爽型, ）,..."


In [64]:
# Add categories to products
sub_type = []   # Subcategory
main_type = []  # Main category
basic_config_data = """护肤品	套装	套装							
护肤品	乳液类	乳液	美白乳	润肤乳	凝乳	柔肤液'	亮肤乳	菁华乳	修护乳
护肤品	眼部护理	眼霜	眼部精华	眼膜					
护肤品	面膜类	面膜													
护肤品	清洁类	洗面	洁面	清洁	卸妆	洁颜	洗颜	去角质	磨砂						
护肤品	化妆水	化妆水	爽肤水	柔肤水	补水露	凝露	柔肤液	精粹水	亮肤水	润肤水	保湿水	菁华水	保湿喷雾	舒缓喷雾
护肤品	面霜类	面霜	日霜	晚霜	柔肤霜	滋润霜	保湿霜	凝霜	日间霜	晚间霜	乳霜	修护霜	亮肤霜	底霜	菁华霜
护肤品	精华类	精华液	精华水	精华露	精华素										
护肤品	防晒类	防晒霜	防晒喷雾												
化妆品	口红类	唇釉	口红	唇彩											
化妆品	底妆类	散粉	蜜粉	粉底液	定妆粉 	气垫	粉饼	BB	CC	遮瑕	粉霜	粉底膏	粉底霜		
化妆品	眼部彩妆	眉粉	染眉膏	眼线	眼影	睫毛膏									
化妆品	修容类	鼻影	修容粉	高光	腮红										
其他	其他	其他"""

# 1. First, define a map collection.
category_config_map = {}
# 2. loop through each row of data
for config_line in basic_config_data.split('\n'):
    # Split each column in this row again
    basic_cateogry_list = config_line.strip().strip('\n').strip('\t').split('\t')
    # print(basic_cateogry_list)
    # Take the first column
    main_category = basic_cateogry_list[0]
    # print(main_category)
    # Take the second column
    sub_category = basic_cateogry_list[1]
    # print(sub_category)
    # Start from the last column and take the last 2 columns
    unit_category_list = basic_cateogry_list[2:]
    # print(unit_category_list)
    # Loop through the unit_category_list
    for unit_category in unit_category_list:        
        unit_category=unit_category.strip('\t').strip('')
            # Assemble into a map collection
        category_config_map[unit_category] = (main_category,sub_category)

In [65]:
category_config_map

{'套装': ('护肤品', '套装'),
 '乳液': ('护肤品', '乳液类'),
 '美白乳': ('护肤品', '乳液类'),
 '润肤乳': ('护肤品', '乳液类'),
 '凝乳': ('护肤品', '乳液类'),
 "柔肤液'": ('护肤品', '乳液类'),
 '亮肤乳': ('护肤品', '乳液类'),
 '菁华乳': ('护肤品', '乳液类'),
 '修护乳': ('护肤品', '乳液类'),
 '眼霜': ('护肤品', '眼部护理'),
 '眼部精华': ('护肤品', '眼部护理'),
 '眼膜': ('护肤品', '眼部护理'),
 '面膜': ('护肤品', '面膜类'),
 '洗面': ('护肤品', '清洁类'),
 '洁面': ('护肤品', '清洁类'),
 '清洁': ('护肤品', '清洁类'),
 '卸妆': ('护肤品', '清洁类'),
 '洁颜': ('护肤品', '清洁类'),
 '洗颜': ('护肤品', '清洁类'),
 '去角质': ('护肤品', '清洁类'),
 '磨砂': ('护肤品', '清洁类'),
 '化妆水': ('护肤品', '化妆水'),
 '爽肤水': ('护肤品', '化妆水'),
 '柔肤水': ('护肤品', '化妆水'),
 '补水露': ('护肤品', '化妆水'),
 '凝露': ('护肤品', '化妆水'),
 '柔肤液': ('护肤品', '化妆水'),
 '精粹水': ('护肤品', '化妆水'),
 '亮肤水': ('护肤品', '化妆水'),
 '润肤水': ('护肤品', '化妆水'),
 '保湿水': ('护肤品', '化妆水'),
 '菁华水': ('护肤品', '化妆水'),
 '保湿喷雾': ('护肤品', '化妆水'),
 '舒缓喷雾': ('护肤品', '化妆水'),
 '面霜': ('护肤品', '面霜类'),
 '日霜': ('护肤品', '面霜类'),
 '晚霜': ('护肤品', '面霜类'),
 '柔肤霜': ('护肤品', '面霜类'),
 '滋润霜': ('护肤品', '面霜类'),
 '保湿霜': ('护肤品', '面霜类'),
 '凝霜': ('护肤品', '面霜类'),
 '日间霜': ('护肤品', '面霜类'),
 '晚间

In [66]:
# Loop through each record in the dataset
for i in range(len(data)):
    exist = False
    # Extract the value of each record in the item_name_cut column
    for temp in data.item_name_cut[i]:
        # Match with the categories in the map; if they exist
        if temp in category_config_map:
            # Then match the corresponding subclass
            sub_type.append(category_config_map.get(temp)[1])
            # Then match the corresponding main class
            main_type.append(category_config_map.get(temp)[0])
            # The marked category exists.
            exist = True
            break
    # If it does not exist, mark it as "others"
    if not exist:
        sub_type.append('其他')
        main_type.append('其他')

In [67]:
print(len(sub_type),len(main_type),len(data))

27512 27512 27512


In [68]:
# Add the subcategory sub_type and the main category main-type as a new column
data['sub_type'] = sub_type
data['main_type'] = main_type

In [69]:
data['sub_type'].value_counts()

sub_type
其他      8085
套装      3465
清洁类     2639
面霜类     2443
化妆水     1774
底妆类     1716
面膜类     1551
乳液类     1226
眼部彩妆    1151
眼部护理    1111
口红类      788
精华类      783
防晒类      494
修容类      286
Name: count, dtype: int64

In [70]:
data['main_type'].value_counts()

main_type
护肤品    15486
其他      8085
化妆品     3941
Name: count, dtype: int64

### (2) Add "Whether it is for men only" as a new column

In [71]:
gender = []
for i in range(len(data)):
    if '男' in data.title[i]:
        gender.append('是')   
    else:
        gender.append('否')

In [72]:
# Add "Is it for men only?" as a new column
data['是否男士专用'] = gender
# Check the data distribution of this column
data['是否男士专用'].value_counts()

是否男士专用
否    24942
是     2570
Name: count, dtype: int64

### (3) Add new sales amount and purchase date (day) as a column

In [73]:
# Sales volume = Sales quantity * Price
data['销售额'] = data.sale_count*data.price

In [74]:
# Convert time format
data['update_time'] = pd.to_datetime(data['update_time'])
# Check
data['update_time']

0       2016-11-14
1       2016-11-14
2       2016-11-14
3       2016-11-14
4       2016-11-14
           ...    
27507   2016-11-05
27508   2016-11-05
27509   2016-11-05
27510   2016-11-05
27511   2016-11-05
Name: update_time, Length: 27512, dtype: datetime64[ns]

In [75]:
# Set the time as the new index
data = data.set_index('update_time')

In [76]:
data['day'] = data.index.day

In [77]:
# Delete the column of Chinese word segmentation
del data['item_name_cut']

### (4) View the final data table

In [79]:
data.head()

,id,title,price,sale_count,comment_count,店名,sub_type,main_type,是否男士专用,销售额,day
update_time,,,,,,,,,,,
2016-11-14,A18164178225,CHANDO/自然堂 雪域精粹纯粹滋润霜50g 补水保湿 滋润水润面霜,139.0,26719.0,2704.0,自然堂,面霜类,护肤品,否,3713941.0,14
2016-11-14,A18177105952,CHANDO/自然堂凝时鲜颜肌活乳液120ML 淡化细纹补水滋润专柜正品,194.0,8122.0,1492.0,自然堂,乳液类,护肤品,否,1575668.0,14
2016-11-14,A18177226992,CHANDO/自然堂活泉保湿修护精华水（滋润型135ml 补水控油爽肤水,99.0,12668.0,589.0,自然堂,化妆水,护肤品,否,1254132.0,14
2016-11-14,A18178033846,CHANDO/自然堂 男士劲爽控油洁面膏 100g 深层清洁 男士洗面奶,38.0,25805.0,4287.0,自然堂,清洁类,护肤品,是,980590.0,14
2016-11-14,A18178045259,CHANDO/自然堂雪域精粹纯粹滋润霜（清爽型）50g补水保湿滋润霜,139.0,5196.0,618.0,自然堂,面霜类,护肤品,否,722244.0,14


In [80]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27512 entries, 2016-11-14 to 2016-11-05
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             27512 non-null  object 
 1   title          27512 non-null  object 
 2   price          27512 non-null  float64
 3   sale_count     27512 non-null  float64
 4   comment_count  27512 non-null  float64
 5   店名             27512 non-null  object 
 6   sub_type       27512 non-null  object 
 7   main_type      27512 non-null  object 
 8   是否男士专用         27512 non-null  object 
 9   销售额            27512 non-null  float64
 10  day            27512 non-null  int32  
dtypes: float64(4), int32(1), object(6)
memory usage: 2.4+ MB


In [85]:
# 保存清理好的数据为Excel格式
data.to_excel('data/clean_beautyProducts.xlsx', sheet_name='clean_data')